In [1]:
from pytqs import tqs
import pandas as pd
import numpy as np

from datetime import date, timedelta
import datetime; 

from sklearn import neighbors
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report

import bytedtqs

In [2]:
appId = "eL38NWjpAMXlC0ifCoo71wk27zE6CS9YQismEcwVH5oNKbJf"
appKey = "VeeOKtXLxjGWE2OyIGkuqWRT33isyxtHH1xlxakeFzId6SMB"
user_id = 'wuweiwei.1012'

## 获取原始数据

In [3]:
sql_txt = """ 

set tqs.query.result.max.lines = 1000000000;

select * from dm_eo_test.team_user_info_period2 limit 1000000000
"""

In [4]:
client = tqs.TQSClient(appId=appId, appKey=appKey)
job = client.execute_query(username = user_id , sql = sql_txt)
data = pd.read_csv(job['resultUrl'])

[2021-03-05 11:24:43,000] - [INFO] - tqs_task_id: 213964822, engine: Hive, status: Created, progress : Submitting
[2021-03-05 11:24:45,031] - [INFO] - tqs_task_id: 213964822, execute id: tqs_213964822-1614914683223
[2021-03-05 11:24:45,032] - [INFO] - application tracking url: http://presto-history.byted.org/proxy/20210305_032443_35593_default
[2021-03-05 11:24:45,033] - [INFO] - tqs_task_id: 213964822, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 11:24:47,068] - [INFO] - tqs_task_id: 213964822, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 11:24:49,089] - [INFO] - tqs_task_id: 213964822, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 11:24:51,118] - [INFO] - tqs_task_id: 213964822, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 11:24:53,179] - [INFO] - tqs_task_id: 213964822, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 11:24:55,201] - [INFO] - tqs_task_id: 213964822, engine: 

[2021-03-05 11:29:06,692] - [INFO] - tqs_task_id: 213964822, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 11:29:11,719] - [INFO] - tqs_task_id: 213964822, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 11:29:16,800] - [INFO] - tqs_task_id: 213964822, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 11:29:21,839] - [INFO] - tqs_task_id: 213964822, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 11:29:26,870] - [INFO] - tqs_task_id: 213964822, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 11:29:32,597] - [INFO] - tqs_task_id: 213964822, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 11:29:37,617] - [INFO] - tqs_task_id: 213964822, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 11:29:42,645] - [INFO] - tqs_task_id: 213964822, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 11:29:47,724] - [INFO] - tqs_task_id: 213964822, eng

In [5]:
data.head()

,user_id,study_dt,active_day_gap,first_milestone_name,occupation,vip_lfc_video,vip_lfc_audio,study_flag,first_team_user,user_group,study_duration,is_clock,study_duration_pre21d,study_duration_post21d,clock_pre21d,clock_post21d
0,2149476050,2020-09-01,561,C2,unknown,NaN,NaN,high,0,Control,4251,1,4251,160969,1,21
1,2149476050,2020-09-02,562,C2,unknown,NaN,NaN,high,0,Control,8323,1,12574,160482,2,21
2,2149476050,2020-09-03,563,C2,unknown,NaN,NaN,high,0,Control,5150,1,17724,163858,3,21
3,2149476050,2020-09-04,564,C2,unknown,NaN,NaN,high,0,Control,6337,1,24061,168633,4,21
4,2149476050,2020-09-05,565,C2,unknown,NaN,NaN,high,0,Control,7620,1,31681,173769,5,21


In [6]:
data["study_duration_pre21d_new"] = data["study_duration_pre21d"] - data["study_duration"]
data["clock_pre21d_new"] = data["clock_pre21d"] - data["is_clock"]
data["first_milestone_name"] = data["first_milestone_name"].fillna("未定级")
data["vip_lfc_video"] = data["vip_lfc_video"].fillna(-1)
data["vip_lfc_audio"] = data["vip_lfc_audio"].fillna(-1)

data.head()

,user_id,study_dt,active_day_gap,first_milestone_name,occupation,vip_lfc_video,vip_lfc_audio,study_flag,first_team_user,user_group,study_duration,is_clock,study_duration_pre21d,study_duration_post21d,clock_pre21d,clock_post21d,study_duration_pre21d_new,clock_pre21d_new
0,2149476050,2020-09-01,561,C2,unknown,-1.0,-1.0,high,0,Control,4251,1,4251,160969,1,21,0,0
1,2149476050,2020-09-02,562,C2,unknown,-1.0,-1.0,high,0,Control,8323,1,12574,160482,2,21,4251,1
2,2149476050,2020-09-03,563,C2,unknown,-1.0,-1.0,high,0,Control,5150,1,17724,163858,3,21,12574,2
3,2149476050,2020-09-04,564,C2,unknown,-1.0,-1.0,high,0,Control,6337,1,24061,168633,4,21,17724,3
4,2149476050,2020-09-05,565,C2,unknown,-1.0,-1.0,high,0,Control,7620,1,31681,173769,5,21,24061,4


In [7]:
len(data)

33740910

## 匹配Test & Control

### 分离Data

In [8]:
test_data = data[data.user_group == 'Test']
control_data = data[(data.user_group == 'Control') & (data.first_team_user == 0)]
len(test_data)

12813

In [9]:
test_data.head()

,user_id,study_dt,active_day_gap,first_milestone_name,occupation,vip_lfc_video,vip_lfc_audio,study_flag,first_team_user,user_group,study_duration,is_clock,study_duration_pre21d,study_duration_post21d,clock_pre21d,clock_post21d,study_duration_pre21d_new,clock_pre21d_new
5725,13232804946,2020-09-12,571,B1高,unknown,-1.0,-1.0,low,1,Test,0,0,6649,4394,4,2,6649,4
13796,78698270360,2020-11-16,46,B1初,office_worker,-1.0,6.0,high,1,Test,2252,1,21525,19245,6,9,19273,5
19657,92963295906,2020-11-21,642,未定级,unknown,-1.0,343.0,low,1,Test,5101,1,5101,15612,1,4,0,0
24079,19671441508,2020-09-11,568,未定级,unknown,-1.0,-1.0,high,1,Test,2892,1,16801,25523,9,15,13909,8
24813,51923900200,2020-10-04,662,未定级,unknown,-1.0,-1.0,low,1,Test,2843,1,20196,8974,6,3,17353,5


In [10]:
match_data = pd.DataFrame(columns = ["study_flag", "user_id_t","clock_post21d_t", "clock_post21d_c", \
                                     "study_duration_post21d_t", "study_duration_post21d_c"])

#### KNN match

In [11]:
test_num = len(test_data)
match_var = ["vip_lfc_video", "vip_lfc_audio", "study_duration_pre21d_new", "clock_pre21d_new"]
metric_var = ["clock_post21d", "study_duration_post21d"]

In [12]:
for index in range(test_num):
    # test user info
    test_data_user = test_data.iloc[index]
    user_id_t = test_data_user.user_id
    study_duration_post21d_t = test_data_user.study_duration_post21d
    clock_post21d_t = test_data_user.clock_post21d
    study_flag = test_data_user.study_flag
    
    join_dt = test_data_user.study_dt
    first_milestone_name = test_data_user.first_milestone_name
    occupation = test_data_user.occupation
    
    
    test_x = test_data_user[match_var]

    # control pool & well match
    control_pool = control_data[(control_data.study_dt == join_dt) & \
                                (control_data.first_milestone_name == first_milestone_name) & \
                                (control_data.occupation == occupation)]

    control_x = control_pool[match_var]
    control_y = control_pool[metric_var]

    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(control_x.to_numpy())

    test_x_list = []  
    test_x_list.append(test_x.to_numpy())
    distances, indices = nbrs.kneighbors(test_x_list)

    control_index = indices[0][0]

    study_duration_post21d_c = control_pool.study_duration_post21d.iloc[control_index]
    clock_post21d_c = control_pool.clock_post21d.iloc[control_index]

    # record paired tc
    match_data = match_data.append({"study_flag": study_flag, "user_id_t": user_id_t, \
                                    "clock_post21d_t": clock_post21d_t, "clock_post21d_c": clock_post21d_c,\
                                    "study_duration_post21d_t": study_duration_post21d_t, \
                                    "study_duration_post21d_c": study_duration_post21d_c}, \
                                   ignore_index = True)
    print('------------' + str(index) + ' ended at ' + str(datetime.datetime.now()) + '------------')
    match_data.to_csv(r'./match_data_p2.csv', encoding='utf_8_sig')

------------0 ended at 2021-03-05 11:36:13.299411------------
------------1 ended at 2021-03-05 11:36:19.819618------------
------------2 ended at 2021-03-05 11:36:27.774892------------
------------3 ended at 2021-03-05 11:36:36.752308------------
------------4 ended at 2021-03-05 11:36:45.404589------------
------------5 ended at 2021-03-05 11:36:52.369275------------
------------6 ended at 2021-03-05 11:37:21.648122------------
------------7 ended at 2021-03-05 11:37:54.808926------------
------------8 ended at 2021-03-05 11:38:01.629959------------
------------9 ended at 2021-03-05 11:38:08.414178------------
------------10 ended at 2021-03-05 11:38:15.242786------------
------------11 ended at 2021-03-05 11:38:21.995148------------
------------12 ended at 2021-03-05 11:38:28.797988------------
------------13 ended at 2021-03-05 11:38:37.379639------------
------------14 ended at 2021-03-05 11:38:44.063772------------
------------15 ended at 2021-03-05 11:38:51.003687------------
--

------------130 ended at 2021-03-05 11:57:29.240812------------
------------131 ended at 2021-03-05 11:57:36.209943------------
------------132 ended at 2021-03-05 11:57:44.872610------------
------------133 ended at 2021-03-05 11:57:53.407497------------
------------134 ended at 2021-03-05 11:58:01.820578------------
------------135 ended at 2021-03-05 11:58:21.022747------------
------------136 ended at 2021-03-05 11:58:40.686367------------
------------137 ended at 2021-03-05 11:58:59.970065------------
------------138 ended at 2021-03-05 11:59:06.686041------------
------------139 ended at 2021-03-05 11:59:14.536264------------
------------140 ended at 2021-03-05 11:59:23.141107------------
------------141 ended at 2021-03-05 11:59:29.857813------------
------------142 ended at 2021-03-05 11:59:37.866386------------
------------143 ended at 2021-03-05 11:59:44.692998------------
------------144 ended at 2021-03-05 11:59:53.219710------------
------------145 ended at 2021-03-05 12:0

------------259 ended at 2021-03-05 12:18:18.121080------------
------------260 ended at 2021-03-05 12:18:26.695423------------
------------261 ended at 2021-03-05 12:18:33.323030------------
------------262 ended at 2021-03-05 12:18:41.312831------------
------------263 ended at 2021-03-05 12:19:01.433053------------
------------264 ended at 2021-03-05 12:19:09.945275------------
------------265 ended at 2021-03-05 12:19:20.949053------------
------------266 ended at 2021-03-05 12:19:27.836087------------
------------267 ended at 2021-03-05 12:19:36.318823------------
------------268 ended at 2021-03-05 12:19:56.168942------------
------------269 ended at 2021-03-05 12:20:12.762277------------
------------270 ended at 2021-03-05 12:20:21.186103------------
------------271 ended at 2021-03-05 12:20:28.024036------------
------------272 ended at 2021-03-05 12:20:34.768644------------
------------273 ended at 2021-03-05 12:20:44.938431------------
------------274 ended at 2021-03-05 12:2

------------388 ended at 2021-03-05 12:41:28.865007------------
------------389 ended at 2021-03-05 12:41:37.396902------------
------------390 ended at 2021-03-05 12:41:43.956357------------
------------391 ended at 2021-03-05 12:41:50.466276------------
------------392 ended at 2021-03-05 12:41:57.121381------------
------------393 ended at 2021-03-05 12:42:05.534421------------
------------394 ended at 2021-03-05 12:42:11.981090------------
------------395 ended at 2021-03-05 12:42:44.377106------------
------------396 ended at 2021-03-05 12:42:51.086196------------
------------397 ended at 2021-03-05 12:43:43.076773------------
------------398 ended at 2021-03-05 12:43:50.023820------------
------------399 ended at 2021-03-05 12:43:56.728627------------
------------400 ended at 2021-03-05 12:44:15.940952------------
------------401 ended at 2021-03-05 12:44:22.849973------------
------------402 ended at 2021-03-05 12:44:29.566655------------
------------403 ended at 2021-03-05 12:4

------------524 ended at 2021-03-05 13:06:25.555606------------
------------525 ended at 2021-03-05 13:06:32.260744------------
------------526 ended at 2021-03-05 13:06:51.236031------------
------------527 ended at 2021-03-05 13:07:25.926805------------
------------528 ended at 2021-03-05 13:07:32.587646------------
------------529 ended at 2021-03-05 13:07:39.234957------------
------------530 ended at 2021-03-05 13:07:45.767561------------
------------531 ended at 2021-03-05 13:07:52.384698------------
------------532 ended at 2021-03-05 13:08:11.716574------------
------------533 ended at 2021-03-05 13:08:43.411185------------
------------534 ended at 2021-03-05 13:08:50.494158------------
------------535 ended at 2021-03-05 13:08:58.571400------------
------------536 ended at 2021-03-05 13:09:07.174376------------
------------537 ended at 2021-03-05 13:09:23.342260------------
------------538 ended at 2021-03-05 13:09:31.372156------------
------------539 ended at 2021-03-05 13:0

...

------------4055 ended at 2021-03-05 22:42:57.127339------------
------------4056 ended at 2021-03-05 22:43:03.889112------------
------------4057 ended at 2021-03-05 22:43:10.538963------------
------------4058 ended at 2021-03-05 22:43:17.475749------------
------------4059 ended at 2021-03-05 22:43:25.183531------------
------------4060 ended at 2021-03-05 22:43:32.086274------------
------------4061 ended at 2021-03-05 22:43:38.957781------------
------------4062 ended at 2021-03-05 22:43:45.809916------------
------------4063 ended at 2021-03-05 22:44:04.651515------------
------------4064 ended at 2021-03-05 22:44:11.523229------------
------------4065 ended at 2021-03-05 22:44:18.377849------------
------------4066 ended at 2021-03-05 22:44:25.205227------------
------------4067 ended at 2021-03-05 22:44:32.730839------------
------------4068 ended at 2021-03-05 22:44:39.247288------------
------------4069 ended at 2021-03-05 22:44:46.413538------------
------------4070 ended at

------------4182 ended at 2021-03-05 23:03:16.167704------------
------------4183 ended at 2021-03-05 23:03:22.850038------------
------------4184 ended at 2021-03-05 23:03:29.426659------------
------------4185 ended at 2021-03-05 23:04:01.071067------------
------------4186 ended at 2021-03-05 23:04:21.258344------------
------------4187 ended at 2021-03-05 23:04:27.984656------------
------------4188 ended at 2021-03-05 23:04:34.654191------------
------------4189 ended at 2021-03-05 23:04:41.312717------------
------------4190 ended at 2021-03-05 23:04:47.885523------------
------------4191 ended at 2021-03-05 23:05:18.803574------------
------------4192 ended at 2021-03-05 23:05:25.566572------------
------------4193 ended at 2021-03-05 23:05:55.476417------------
------------4194 ended at 2021-03-05 23:06:03.615579------------
------------4195 ended at 2021-03-05 23:06:12.017124------------
------------4196 ended at 2021-03-05 23:06:31.418848------------
------------4197 ended at

------------4320 ended at 2021-03-05 23:26:54.776850------------
------------4321 ended at 2021-03-05 23:27:01.489489------------
------------4322 ended at 2021-03-05 23:27:08.064253------------
------------4323 ended at 2021-03-05 23:27:14.873386------------
------------4324 ended at 2021-03-05 23:27:21.547627------------
------------4325 ended at 2021-03-05 23:27:28.350550------------
------------4326 ended at 2021-03-05 23:27:47.518622------------
------------4327 ended at 2021-03-05 23:28:03.291809------------
------------4328 ended at 2021-03-05 23:28:13.662382------------
------------4329 ended at 2021-03-05 23:28:20.111999------------
------------4330 ended at 2021-03-05 23:28:26.798893------------
------------4331 ended at 2021-03-05 23:28:35.169471------------
------------4332 ended at 2021-03-05 23:28:42.111219------------
------------4333 ended at 2021-03-05 23:28:48.859833------------
------------4334 ended at 2021-03-05 23:28:55.616103------------
------------4335 ended at

------------4460 ended at 2021-03-05 23:49:37.244240------------
------------4461 ended at 2021-03-05 23:49:43.833896------------
------------4462 ended at 2021-03-05 23:49:51.388884------------
------------4463 ended at 2021-03-05 23:49:59.562516------------
------------4464 ended at 2021-03-05 23:50:06.207952------------
------------4465 ended at 2021-03-05 23:50:13.060709------------
------------4466 ended at 2021-03-05 23:50:32.515579------------
------------4467 ended at 2021-03-05 23:50:41.014032------------
------------4468 ended at 2021-03-05 23:50:48.388951------------
------------4469 ended at 2021-03-05 23:50:54.770040------------
------------4470 ended at 2021-03-05 23:51:10.610209------------
------------4471 ended at 2021-03-05 23:51:17.381370------------
------------4472 ended at 2021-03-05 23:51:25.564449------------
------------4473 ended at 2021-03-05 23:51:32.353156------------
------------4474 ended at 2021-03-05 23:51:38.979424------------
------------4475 ended at

ValueError: Found array with 0 sample(s) (shape=(0, 4)) while a minimum of 1 is required.